In [1]:
import pandas as pd
#working on integrating this information with mtgjson, but until that feature works, we don't need this.
#import utility


In [2]:
#path to our data, feel free to change this to whatever dataset you want.
buylistPath = 'Buylist.csv'
pricingPath = 'Pricing.csv'

#Fees for buying and selling. Loosely based on TCGplayer numbers.
#Will be a little more for cheaper cards, and a little less for more expensive cards
sellFee = .12
buyFee = .10

#for the simple math we've got set up, this makes it easier down below.
sellCut = 1 - sellFee
buyCut = 1 - buyFee

#If I can't buy a cheeseburger with something, it doesn't actually have value.
cheeseburger = 5

In [3]:
buylistData = pd.read_csv(buylistPath)
pricingData = pd.read_csv(pricingPath)

In [4]:
#make some dataframes that are smaller, so that we can run equivalencies on them, because sometimes the data is bad.
buylist = buylistData[['TCGplayer Id','Set Name', 'Product Name']]
pricelist = pricingData[['TCGplayer Id','Set Name', 'Product Name']]

#checking if our data is clean.
#validates that tcgplayer id correlates to the same Set and Product Name
pricelist = pricelist.sort_values('TCGplayer Id')
pricelist = pricelist.reset_index(drop=True)

buylist = buylist.sort_values('TCGplayer Id')
buylist = buylist.reset_index(drop=True)

pricelist.compare(buylist)

Empty DataFrame
Columns: []
Index: []

In [5]:
#we know that our small data is clean, so now we can do the same to the big data.
pricelist = pricingData.sort_values('TCGplayer Id')
pricelist = pricelist.reset_index(drop=True)

buylist = buylistData.sort_values('TCGplayer Id')
buylist = buylist.reset_index(drop=True)

In [6]:
#getting the columns to use as our key values for the merge
#if we don't do this, we get a bunch of columns we have to remove.
#I don't know why pandas supports difference, but not unions on these things.
set1 = set(pricelist.columns.to_list())
set2 = set(buylist.columns.to_list())

keys = list(set1.intersection(set2))

result = pd.merge(pricelist, buylist, on=keys, how='outer')

In [7]:
#drop the utility columns that we don't care about.
result = result.drop(['Title', 'Add to Quantity', 'TCG Marketplace Price', 'My Store Reserve Quantity', 'My Store Price', 'Photo URL', 
'Total Quantity', 'Add to Quantity', 'TCG Marketplace Price', 'My Store Reserve Quantity', 'My Store Price', 'Buylist Quantity', 
'Add to Buylist Quantity', 'My Buylist Price', 'Pending Purchase Quantity', 'Number'], axis = 1)

In [8]:
#naively, we can now compare buylist market/high to tcg market/low, and get cards that make money.
#we'll do this only on things that have values for all of that.
#TODO: change this to filters in case the number of tables changes
filtered = result.dropna(thresh=12).copy()

In [9]:
filtered.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price
15,363363,Magic,Portal Second Age,Archangel,R,Lightly Played,3.95,4.00,4.94,3.95,2.30,1.42
16,363364,Magic,Portal Second Age,Armageddon,R,Lightly Played,8.80,8.40,8.40,8.40,5.64,5.22
29,363377,Magic,Portal Second Age,Coastal Wizard,R,Lightly Played,2.64,2.50,3.39,2.40,1.11,0.89
54,363402,Magic,Portal Second Age,Goblin Lore,U,Lightly Played,1.82,1.09,1.99,1.00,1.00,0.84
55,363403,Magic,Portal Second Age,Goblin Matron,U,Lightly Played,2.78,1.28,1.99,1.00,1.26,0.94


In [10]:
filtered.shape

(16859, 12)

In [11]:
#creating new columns! All of this information is derived, so should maybe be put into functions?
#sellCut and buyCut are defined in cell 2

#Optimistic is buy market, sell market.
#Pessimistic is buy high, sell low
#Emulating high velocity is buy market, sell low
#Buy High Sell Market is when you think not enough people in general are selling something? Unsure why you would use it, but it's here.

filtered['Market-Market'] = filtered['TCG Market Price']*sellCut - filtered['Buylist Market Price']*buyCut
filtered['Market-High'] = filtered['TCG Market Price']*sellCut - filtered['Buylist High Price']*buyCut
filtered['Low-Market'] = filtered['TCG Low Price']*sellCut - filtered['Buylist Market Price']*buyCut
filtered['Low-High'] = filtered['TCG Low Price']*sellCut - filtered['Buylist High Price']*buyCut

In [12]:
filtered.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
15,363363,Magic,Portal Second Age,Archangel,R,Lightly Played,3.95,4.00,4.94,3.95,2.30,1.42,1.4060,2.1980,1.406,2.198
16,363364,Magic,Portal Second Age,Armageddon,R,Lightly Played,8.80,8.40,8.40,8.40,5.64,5.22,2.6680,3.0460,2.316,2.694
29,363377,Magic,Portal Second Age,Coastal Wizard,R,Lightly Played,2.64,2.50,3.39,2.40,1.11,0.89,1.3242,1.5222,1.113,1.311
54,363402,Magic,Portal Second Age,Goblin Lore,U,Lightly Played,1.82,1.09,1.99,1.00,1.00,0.84,0.7016,0.8456,-0.020,0.124
55,363403,Magic,Portal Second Age,Goblin Matron,U,Lightly Played,2.78,1.28,1.99,1.00,1.26,0.94,1.3124,1.6004,-0.254,0.034


In [13]:
#this is kind of a loaded name, but basically it's where the market makes sense. If the market price is lower than the lowest price, the market price is probably wrong.
#at the very least, something is weird, and we don't want to have a computer make a decision about weird things.
rational = filtered.query('(`Buylist High Price` > `Buylist Market Price` and `TCG Market Price` > `TCG Low Price`)')

In [14]:
rational.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
118,363466,Magic,Portal Second Age,Sleight of Hand,C,Lightly Played,3.67,4.46,4.49,3.50,1.35,1.52,2.0146,1.8616,1.8650,1.7120
135,363483,Magic,Portal Second Age,Temporal Manipulation,R,Lightly Played,34.10,27.00,26.99,26.00,18.37,19.34,13.4750,12.6020,6.3470,5.4740
188,363536,Magic,Starter 1999,Devastation,R,Lightly Played,16.90,15.07,15.07,15.07,11.09,11.70,4.8910,4.3420,3.2806,2.7316
283,363631,Magic,Starter 1999,Summer Bloom,R,Lightly Played,2.35,1.92,2.91,1.92,0.72,0.95,1.4200,1.2130,1.0416,0.8346
470,363818,Magic,Portal Three Kingdoms,Zodiac Dog,C,Lightly Played,12.80,12.83,11.49,11.49,5.93,6.21,5.9270,5.6750,4.7742,4.5222


In [15]:
#How many cards behave the way do we expect, and what is the size of the data set we want to act on?
rational.shape

(2746, 16)

In [16]:
#And with that, the data is mostly there for you to peruse!

#Some examples!
#if optimistically, we're making less than $1, that's bad!
loss = rational.query('`Market-Market` <= 1')
#if pessimistically, we're making more than $5, that's okay!
win = rational.query('`Low-High` >= 5')

In [17]:
#How many wins do we have?
win.shape

(189, 16)

In [18]:
#What are the top wins?
win.sort_values('Market-Market')
win.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
135,363483,Magic,Portal Second Age,Temporal Manipulation,R,Lightly Played,34.10,27.00,26.99,26.00,18.37,19.34,13.4750,12.6020,6.3470,5.4740
522,363870,Magic,Portal,Cruel Tutor,R,Lightly Played,29.24,32.85,28.49,27.50,19.28,19.38,8.3792,8.2892,6.8480,6.7580
2850,366220,Magic,Arabian Nights,Island Fish Jasconius,R,Lightly Played,33.88,69.90,29.99,29.99,18.01,18.06,13.6054,13.5604,10.1822,10.1372
2864,366235,Magic,Arabian Nights,Mijae Djinn,R,Lightly Played,38.64,39.15,38.00,38.00,22.17,23.33,14.0502,13.0062,13.4870,12.4430
2880,366251,Magic,Arabian Nights,Shahrazad,R,Lightly Played,432.40,699.00,399.98,399.98,286.80,287.02,122.3920,122.1940,93.8624,93.6644


In [19]:
#How many losses?
loss.shape

(1188, 16)

In [20]:
loss.sort_values('Low-High')
loss.head()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
534,363882,Magic,Portal,Earthquake,R,Lightly Played,1.73,1.50,2.61,1.62,0.62,0.71,0.9644,0.8834,0.8676,0.7866
597,363945,Magic,Portal,Nature's Lore,C,Lightly Played,1.83,2.73,2.32,1.33,1.01,1.04,0.7014,0.6744,0.2614,0.2344
688,364036,Magic,Unglued,Bureaucracy,R,Lightly Played,0.84,0.98,1.65,0.66,0.28,0.29,0.4872,0.4782,0.3288,0.3198
739,364087,Magic,Unglued,"Look at Me, I'm the DCI",R,Lightly Played,0.61,0.51,1.49,0.50,0.16,0.18,0.3928,0.3748,0.2960,0.2780
741,364089,Magic,Unglued,"Mine, Mine, Mine!",R,Lightly Played,0.97,1.23,1.87,0.88,0.31,0.35,0.5746,0.5386,0.4954,0.4594


In [21]:
#What if we look at all the things that make no sense?
strange = filtered.query('`TCG Market Price` < `TCG Low Price`')
strange.sample(10)

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
108028,7139218,Magic,March of the Machine: The Aftermath,"Calix, Guided by Fate",M,Lightly Played Foil,12.31,58.50,17.93,16.44,3.98,6.11,7.2508,5.3338,10.8852,8.9682
17290,405990,Magic,Oversize Cards,The Great Forest (Planechase),S,Lightly Played,2.66,2.50,5.02,3.03,0.66,0.19,1.7468,2.1698,2.0724,2.4954
3604,366985,Magic,Legends,Winds of Change,U,Lightly Played,18.83,22.13,19.33,19.33,14.58,13.26,3.4484,4.6364,3.8884,5.0764
27579,844615,Magic,Premium Deck Series: Graveborn,Reanimate,U,Lightly Played Foil,16.04,16.14,16.14,16.14,10.46,10.78,4.7012,4.4132,4.7892,4.5012
92932,5652656,Magic,Secret Lair Drop Series,"Sisay, Weatherlight Captain",R,Lightly Played,4.13,5.99,5.99,5.99,2.59,2.45,1.3034,1.4294,2.9402,3.0662
23834,839315,Magic,Judge Promos,Mind's Desire,P,Lightly Played Foil,16.58,59.98,39.99,39.99,9.84,10.61,5.7344,5.0414,26.3352,25.6422
62938,4131908,Magic,Prerelease Cards,Cavalier of Thorns,M,Lightly Played Foil,17.30,18.94,18.95,18.95,9.91,10.00,6.3050,6.2240,7.7570,7.6760
39736,1270378,Magic,Commander 2014,Comeuppance,R,Lightly Played,7.36,7.75,7.75,7.75,6.62,4.80,0.5188,2.1568,0.8620,2.5000
94224,5702783,Magic,Double Masters 2022,Force of Negation (Borderless),R,Lightly Played,36.48,39.96,41.05,39.96,24.16,25.26,10.3584,9.3684,13.4208,12.4308
86001,5302789,Magic,Commander: Innistrad: Crimson Vow,Vandalblast,U,Lightly Played,4.54,4.99,5.00,5.00,2.82,2.02,1.4572,2.1772,1.8620,2.5820


In [22]:
#Looking at all the strange cards in a set
strange.query('`Set Name` == "Dominaria"')

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
54724,3613955,Magic,Dominaria,Haphazard Bombardment,R,Lightly Played Foil,0.12,0.14,1.13,0.14,0.01,0.02,0.0966,0.0876,0.1142,0.1052
54782,3614245,Magic,Dominaria,The Eldest Reborn,U,Lightly Played Foil,8.55,9.87,9.50,9.50,4.63,4.90,3.3570,3.1140,4.1930,3.9500
54860,3614635,Magic,Dominaria,"Multani, Yavimaya's Avatar",M,Lightly Played Foil,4.69,5.49,5.00,5.00,2.46,2.58,1.9132,1.8052,2.1860,2.0780
54905,3614860,Magic,Dominaria,"Karn, Scion of Urza",M,Lightly Played,6.95,8.69,7.05,7.05,4.58,4.02,1.9940,2.4980,2.0820,2.5860
55212,3634087,Magic,Dominaria,Weatherlight,M,Lightly Played Foil,4.74,4.24,5.00,5.00,2.78,2.30,1.6692,2.1012,1.8980,2.3300
55334,3635615,Magic,Dominaria,Cabal Stronghold,R,Lightly Played Foil,7.22,7.49,7.49,7.49,3.81,4.20,2.9246,2.5736,3.1622,2.8112
55719,3637816,Magic,Dominaria,Shivan Fire,C,Lightly Played Foil,0.24,0.25,1.24,0.25,0.01,0.01,0.2022,0.2022,0.2110,0.2110


In [23]:
#how many things are in our query?
strange.query('`Set Name` == "Dominaria"').shape

(7, 16)

In [24]:
#What are the cards where the difference is more than 30%, and the market price is more than $1?
strange.query('(abs((`TCG Market Price` / `TCG Low Price`) - 1) > .30) and `TCG Market Price` > 1' )

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
975,364331,Magic,Alpha Edition,Rod of Ruin,U,Lightly Played,54.99,81.98,80.00,80.00,57.39,23.96,-3.2598,26.8272,18.7490,48.8360
7412,370793,Magic,Beta Edition,Black Vise,U,Lightly Played,107.99,216.19,194.99,190.00,112.81,91.80,-6.4978,12.4112,65.6710,84.5800
7942,371330,Magic,Unlimited Edition,Vesuvan Doppelganger,R,Lightly Played,113.99,279.94,200.99,200.00,120.78,120.83,-8.3908,-8.4358,67.2980,67.2530
17262,405962,Magic,Oversize Cards,Glimmervoid Basin (Planechase),S,Lightly Played,2.21,1.90,5.26,5.26,0.44,0.20,1.5488,1.7648,4.2328,4.4488
19196,830293,Magic,10th Edition,Caves of Koilos,R,Lightly Played Foil,4.96,7.95,7.95,7.95,8.76,3.00,-3.5192,1.6648,-0.8880,4.2960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100701,6708068,Magic,The Brothers' War,Cityscape Leveler,M,Lightly Played Foil,17.66,48.86,33.00,29.00,8.00,9.94,8.3408,6.5948,18.3200,16.5740
100917,6716921,Magic,The Brothers' War,Cityscape Leveler (Extended Art),M,Lightly Played,18.56,42.82,28.42,28.42,11.63,13.03,5.8658,4.6058,14.5426,13.2826
103895,6921965,Magic,Secret Lair Drop Series,Muscle Sliver (Extended Art),R,Lightly Played Foil,10.60,15.68,19.24,19.24,14.04,9.50,-3.3080,0.7780,4.2952,8.3812
106643,7076211,Magic,Commander: March of the Machine,Firemane Commando,R,Lightly Played Foil,1.72,4.29,4.29,3.30,0.11,0.68,1.4146,0.9016,2.8050,2.2920


In [25]:
#Market price on a card is more than 50 cents, but nobody is buylisting it! There might be potential here! (but be careful of buylist fees!)
result.query('`TCG Market Price` > .50 and `Buylist High Price`.isnull()').sort_values('TCG Market Price').tail()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price
73712,4601468,Magic,Zendikar Rising,Mountain,L,Lightly Played Foil,0.90,0.96,1.74,0.75,0.25,NaN
30880,850686,Magic,Duel Decks: Venser vs. Koth,Sawtooth Loon,U,Lightly Played,0.99,0.14,1.13,0.14,NaN,NaN
69994,4456002,Magic,Core Set 2021,Liliana's Steward (Showcase),C,Lightly Played Foil,0.99,0.11,1.04,0.05,0.01,NaN
67549,4337883,Magic,Unsanctioned,Spirit of the Season,U,Lightly Played,1.17,NaN,1.12,0.13,NaN,NaN
96997,5815748,Magic,"Universes Beyond: Warhammer 40,000",Dismal Backwater,C,Lightly Played,1.99,0.10,1.15,0.16,0.14,NaN


In [26]:
#Among cards that make sense, mid-range LP Foils!
rational.query('(50 > `TCG Market Price` > 10) and `Condition` == "Lightly Played Foil"').sort_values('Low-High').tail()

,TCGplayer Id,Product Line,Set Name,Product Name,Rarity,Condition,TCG Market Price,TCG Direct Low,TCG Low Price With Shipping,TCG Low Price,Buylist Market Price,Buylist High Price,Market-Market,Market-High,Low-Market,Low-High
93655,5686545,Magic,Secret Lair Drop Series,Tiamat (Showcase),M,Lightly Played Foil,44.97,41.77,43.35,43.35,27.13,27.30,15.1566,15.0036,13.7310,13.5780
21403,833891,Magic,Champs Promos,Voidslime,P,Lightly Played Foil,46.42,44.99,44.98,44.98,25.98,26.91,17.4676,16.6306,16.2004,15.3634
19678,830775,Magic,7th Edition,Intrepid Hero,R,Lightly Played Foil,43.49,41.38,41.38,41.38,23.06,23.14,17.5172,17.4452,15.6604,15.5884
33746,1086068,Magic,Modern Masters,Tarmogoyf,M,Lightly Played Foil,49.23,48.48,48.95,48.95,26.73,27.27,19.2654,18.7794,19.0190,18.5330
25359,841207,Magic,Mercadian Masques,General's Regalia,R,Lightly Played Foil,43.70,39.63,39.61,39.61,17.02,17.55,23.1380,22.6610,19.5388,19.0618
